<a href="https://colab.research.google.com/github/nikhilkoditala/CSC-8980-NLP/blob/main/NLP_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Nikhil Koditala

# Panther ID: 002571023

1. Create a cosine similarity matrix for all of Shakespeare’s works found in the provided file. This will result in a 42 by 42 matrix with the cosine similarity between each of his works. In other words, calculate the document-wise cosine similarity between all of Shakepeare’s works. (50 points). ​Use TF_IDF for this. Note, you can use the Cosine Similarity function on scikit-learn or implement your own, but no other library/package is allowed.


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.datasets import fetch_20newsgroups
import os, random
import numpy as np
import nltk
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
folder_path = '/content/drive/MyDrive/shakespeares-works_TXT_FolgerShakespeare'

In [4]:
txt_files = os.listdir(folder_path)
data = []
for file in txt_files:
   data.append(open(os.path.join(folder_path,file), 'r').read())

In [5]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

In [6]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.90761656 0.77494139 ... 0.81097319 0.59536997 0.66267568]
 [0.90761656 1.         0.78910615 ... 0.83414428 0.6086646  0.67148272]
 [0.77494139 0.78910615 1.         ... 0.83172303 0.59251467 0.65190502]
 ...
 [0.81097319 0.83414428 0.83172303 ... 1.         0.61221239 0.67887912]
 [0.59536997 0.6086646  0.59251467 ... 0.61221239 1.         0.52581905]
 [0.66267568 0.67148272 0.65190502 ... 0.67887912 0.52581905 1.        ]]


2 Write a function that takes the previous matrix and a number ​n​ as parameters (nothing else will be accepted) and return the top ​n​ similar works. Use the function to output the top 10 similar works. (30 points).

In [7]:
def n_similar_works(cosine_sim, N):
  for i in range(len(cosine_sim)):
    cosine_sim[i][i] = 0 # Making diagonal elements as zero, to discard them from sorting algorithm

  index_1d = cosine_sim.flatten().argsort()[-(2*N):]
  x_idx, y_idx = np.unravel_index(index_1d, cosine_sim.shape)
  
  x_idx = np.flip(x_idx)
  y_idx = np.flip(y_idx)

  similar_works = []
  for i in range(0,len(x_idx),2):
    x = x_idx[i]
    y = y_idx[i]
    if(x != y):
      similar_works.append((x,y,cosine_sim[x][y]))
  
  return similar_works

In [8]:
similar_works = n_similar_works(cosine_sim,10)

In [9]:
print('Similar works are:')
for work in similar_works:
  print(txt_files[work[0]],' and ',txt_files[work[1]],' - ',work[2])

Similar works are:
venus-and-adonis_TXT_FolgerShakespeare.txt  and  lucrece_TXT_FolgerShakespeare.txt  -  0.9175430138884013
henry-iv-part-2_TXT_FolgerShakespeare.txt  and  henry-iv-part-1_TXT_FolgerShakespeare.txt  -  0.9076165568984116
shakespeares-sonnets_TXT_FolgerShakespeare.txt  and  lucrece_TXT_FolgerShakespeare.txt  -  0.867768874887524
henry-vi-part-2_TXT_FolgerShakespeare.txt  and  henry-vi-part-1_TXT_FolgerShakespeare.txt  -  0.84379159802984
richard-ii_TXT_FolgerShakespeare.txt  and  richard-iii_TXT_FolgerShakespeare.txt  -  0.838952501010752
shakespeares-sonnets_TXT_FolgerShakespeare.txt  and  venus-and-adonis_TXT_FolgerShakespeare.txt  -  0.8381452108269781
henry-viii_TXT_FolgerShakespeare.txt  and  henry-v_TXT_FolgerShakespeare.txt  -  0.8363042342905281
henry-vi-part-2_TXT_FolgerShakespeare.txt  and  henry-vi-part-3_TXT_FolgerShakespeare.txt  -  0.8354926670186525
richard-ii_TXT_FolgerShakespeare.txt  and  henry-vi-part-2_TXT_FolgerShakespeare.txt  -  0.834350777065286



3 Using the code from the Language Models II class, train two simple language models using all of the files (together) in shakespeares-works_TXT_FolgerShakespeare.zip. One model should be trained using bigrams, the other using trigrams. (40 points).

In [10]:
tokens = []
for doc in data:
  tokens.append(nltk.word_tokenize(doc))

a) Bigrams Model

In [11]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model_bigram = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurence  
for sentence in tokens:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        model_bigram[w1][w2] += 1

# Let's transform the counts to probabilities
for w1 in model_bigram:
    total_count = float(sum(model_bigram[w1].values()))
    for w2 in model_bigram[w1]:
        model_bigram[w1][w2] /= total_count

In [12]:
dict(model_bigram["Trojan"])

{'!': 0.05128205128205128,
 "'s": 0.02564102564102564,
 ',': 0.1282051282051282,
 '.': 0.07692307692307693,
 ';': 0.02564102564102564,
 '?': 0.02564102564102564,
 'And': 0.02564102564102564,
 'and': 0.05128205128205128,
 'ass': 0.02564102564102564,
 'bleeds': 0.02564102564102564,
 'blood': 0.05128205128205128,
 'dead': 0.02564102564102564,
 'did': 0.02564102564102564,
 'drab': 0.02564102564102564,
 'hath': 0.02564102564102564,
 'horse': 0.02564102564102564,
 'is': 0.02564102564102564,
 'leaders': 0.02564102564102564,
 'lords': 0.02564102564102564,
 'mothers': 0.02564102564102564,
 'part': 0.02564102564102564,
 'prisoner': 0.02564102564102564,
 'scorns': 0.02564102564102564,
 'so': 0.02564102564102564,
 'soldiers': 0.02564102564102564,
 'that': 0.02564102564102564,
 'trail': 0.02564102564102564,
 'trumpets': 0.02564102564102564,
 'under': 0.02564102564102564,
 'walls': 0.02564102564102564}

b) Trigrams Model

In [13]:
# Create a placeholder for model
model_trigram = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurence  
for sentence in tokens:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model_trigram[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model_trigram:
    total_count = float(sum(model_trigram[w1_w2].values()))
    for w3 in model_trigram[w1_w2]:
        model_trigram[w1_w2][w3] /= total_count

In [14]:
dict(model_trigram["cruel","battle"])

{'here': 1.0}

 4 Write a function that takes the following three parameters: model, list of start words, number of sentences to generate. This function should return the sentences generated as a list. DO NOT print anything to the screen from within the function. Use this function to generate 10 sentences with the bigram model from the previous question, and 5 sentences with the trigram model from the previous question. (30 points)

In [15]:
def gen_sentences(model,start_words,num_sentences):
  sentences = []
  
  for key in model.keys():
    break
  
  if(type(key) == tuple):
    for i in range(num_sentences):
      text = start_words
      sentence_finished = False

      while not sentence_finished:
        # select a random probability threshold  
        r = random.random()
        accumulator = .0

        for word in model[tuple(text[-2:])].keys():
            accumulator += model[tuple(text[-2:])][word]
            # select words that are above the probability threshold
            if accumulator >= r:
                text.append(word)
                break

        if text[-2:] == [None, None]:
            sentence_finished = True
        
      sentences.append(' '.join([t for t in text if t]))
  else:
    for i in range(num_sentences):
      text = start_words
      sentence_finished = False

      while not sentence_finished:
        # select a random probability threshold  
        r = random.random()
        accumulator = .0

        for word in model[text[-1]].keys():
            accumulator += model[text[-1]][word]
            # select words that are above the probability threshold
            if accumulator >= r:
                text.append(word)
                break

        if text[-1:] == [None]:
            sentence_finished = True
        
      sentences.append(' '.join([t for t in text if t]))

  return sentences

In [16]:
sentences_bigram = gen_sentences(model_bigram,['cruel'],10)

In [17]:
sentences_trigram = gen_sentences(model_trigram,['cruel','battle'],5)

Bonus (20 points): Using the same methodology from questions 1 and 2, create a similarity matrix between the 20 newsgroups corpus. And find the top 5 similar newsgroups.

In [20]:
twenty_data = fetch_20newsgroups()

In [21]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(twenty_data.data)

In [22]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.04405974 0.11017033 ... 0.04433678 0.04457107 0.0329325 ]
 [0.04405974 1.         0.06242113 ... 0.07373268 0.06959306 0.02439956]
 [0.11017033 0.06242113 1.         ... 0.07569182 0.06214891 0.02357985]
 ...
 [0.04433678 0.07373268 0.07569182 ... 1.         0.02909961 0.00716986]
 [0.04457107 0.06959306 0.06214891 ... 0.02909961 1.         0.02428174]
 [0.0329325  0.02439956 0.02357985 ... 0.00716986 0.02428174 1.        ]]


In [25]:
similar_works = n_similar_works(cosine_sim,5)

In [26]:
print(similar_works)

[(10777, 2002, 1.0000000000000002), (5392, 14, 1.0000000000000002), (9989, 800, 1.0), (4495, 4772, 0.9997809868890166), (8665, 4772, 0.9996809512865921)]
